<a href="https://colab.research.google.com/github/onewon1234/AI_DL_Project/blob/KoElectra/KoElectra/train_tuning_2_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import re
import random
import os
from tqdm.auto import tqdm

In [2]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn

In [3]:
from scipy.stats import loguniform
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import itertools
from itertools import permutations
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

In [4]:
print(torch.cuda.is_available())

True


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/강의/AI를 위한 딥러닝/AI_DL_Project/code

/content/drive/MyDrive/강의/AI를 위한 딥러닝/AI_DL_Project/code


In [7]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [8]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7351 entries, 0 to 7350
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          7351 non-null   object
 1   sentence_0  7351 non-null   object
 2   sentence_1  7351 non-null   object
 3   sentence_2  7351 non-null   object
 4   sentence_3  7351 non-null   object
 5   answer_0    7351 non-null   int64 
 6   answer_1    7351 non-null   int64 
 7   answer_2    7351 non-null   int64 
 8   answer_3    7351 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 517.0+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


In [9]:
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          1780 non-null   object
 1   sentence_0  1780 non-null   object
 2   sentence_1  1780 non-null   object
 3   sentence_2  1780 non-null   object
 4   sentence_3  1780 non-null   object
dtypes: object(5)
memory usage: 69.7+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3
0,TEST_0000,"자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐...","결정론은 모든 사건이 원인과 결과의 연쇄에 의해 발생한다고 주장하며, 이는 인간의 ...",그러나 인간의 인식과 선택 과정에서 나타나는 복잡성과 예측 불가능성은 자유 의지의 ...,"결국, 자유 의지와 결정론은 서로를 배제하기보다는, 인간 경험의 다양한 측면을 설명..."
1,TEST_0001,사회적 낙인은 개인의 자아 존중감에 부정적인 영향을 미친다.,"건강 불평등은 이러한 낙인으로 인해 더욱 심화되며, 특정 집단이 의료 서비스 접근에...","결국, 사회적 낙인과 건강 불평등은 서로 연결되어 있으며, 이를 해결하기 위한 포괄...","낙인으로 인해 사람들은 사회적 고립을 경험하고, 이는 정신적 및 신체적 건강에 악영..."
2,TEST_0002,글쓰기 능력을 키우기 위해서는 꾸준한 연습이 필수적이다.,"마지막으로, 독서를 통해 다른 작가들의 기법을 배우는 것은 창의력을 자극하는 데 도...",피드백을 받는 과정은 글의 질을 향상시키는 중요한 요소로 작용한다.,다양한 주제에 대해 글을 써보면 자신의 스타일과 강점을 발견할 수 있다.
3,TEST_0003,작은 공간에서도 효율적으로 사용할 수 있어 집안의 혼잡함을 줄여준다.,정기적으로 내용을 점검하면 필요 없는 물건을 정리할 수 있다.,각 칸을 활용하여 카테고리별로 물건을 나누면 찾기 쉬워진다.,다용도 수납함은 다양한 물건을 정리하는 데 유용하다.
4,TEST_0004,음악은 특정 문화의 가치와 전통을 반영하는 중요한 매체이다.,이러한 음악적 표현은 공동체의 소속감을 증진시키는 역할을 한다.,각 문화는 고유한 음악적 요소를 통해 정체성을 형성하고 강화한다.,"결국, 음악은 개인과 집단의 문화적 정체성을 이해하는 데 필수적인 요소로 작용한다."


In [10]:
submission.info()
submission.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        1780 non-null   object
 1   answer_0  1780 non-null   int64 
 2   answer_1  1780 non-null   int64 
 3   answer_2  1780 non-null   int64 
 4   answer_3  1780 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 69.7+ KB


,ID,answer_0,answer_1,answer_2,answer_3
0,TEST_0000,0,1,2,3
1,TEST_0001,0,1,2,3
2,TEST_0002,0,1,2,3
3,TEST_0003,0,1,2,3
4,TEST_0004,0,1,2,3


In [11]:
# 텍스트 정제
def clean_text(text):
  # 특수문자 제거
  text = re.sub(r'[^\w\s]', '', text)
  # 소문자 변환: 한글에는 무의미
  text = text.lower()
  # 불필요한 공백 제거
  text = ' '.join(text.split())
  return text

In [12]:
# 정제 전
print(train['sentence_0'][0])
print(test['sentence_0'][0])

블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.
자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐구할 가치가 있다.


In [13]:
# 텍스트 정제
for i in range(4):
    train[f'sentence_{i}'] = train[f'sentence_{i}'].apply(clean_text)
    test[f'sentence_{i}'] = test[f'sentence_{i}'].apply(clean_text)

In [14]:
# 정제 후
print(train['sentence_0'][0])
print(test['sentence_0'][0])

블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다
자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만 이 둘의 공존 가능성도 탐구할 가치가 있다


In [15]:
## ✅ Pairwise 전처리 함수
def preprocess_pairwise(df):
    data = []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answer = [row[f'answer_{i}'] for i in range(4)]
        ordered = [sentences[i] for i in answer]
        positive_pairs = [(ordered[i], ordered[i+1]) for i in range(3)]
        all_pairs = list(itertools.permutations(sentences, 2))
        for s1, s2 in all_pairs:
            label = 1 if (s1, s2) in positive_pairs else 0
            data.append({'sentence1': s1, 'sentence2': s2, 'label': label})
    return pd.DataFrame(data)

In [16]:
# 전처리 모듈 사용
sampled_train = train.sample(n=1000, random_state=42).reset_index(drop=True)
pairwise_df = preprocess_pairwise(sampled_train)

In [17]:
pairwise_df.info()
pairwise_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence1  12000 non-null  object
 1   sentence2  12000 non-null  object
 2   label      12000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 281.4+ KB


,sentence1,sentence2,label
0,원격 의료 시스템 구축에는 안정적인 인터넷 인프라가 필수적이다,의료 데이터의 안전한 전송을 위해 암호화 기술이 필요하다,1
1,원격 의료 시스템 구축에는 안정적인 인터넷 인프라가 필수적이다,마지막으로 지속적인 시스템 유지보수와 업데이트가 서비스의 품질을 보장한다,0
2,원격 의료 시스템 구축에는 안정적인 인터넷 인프라가 필수적이다,사용자 친화적인 인터페이스는 환자와 의료진 간의 원활한 소통을 지원한다,0
3,의료 데이터의 안전한 전송을 위해 암호화 기술이 필요하다,원격 의료 시스템 구축에는 안정적인 인터넷 인프라가 필수적이다,0
4,의료 데이터의 안전한 전송을 위해 암호화 기술이 필요하다,마지막으로 지속적인 시스템 유지보수와 업데이트가 서비스의 품질을 보장한다,0


# 2. 데이터셋 구성

In [18]:
MAX_TOKEN_LENGTH = 128

In [19]:
# 데이터셋 클래스 정의
class SentencePairDataset(Dataset):
    """
    문장 쌍을 받아 BERT 입력 형식으로 변환하는 PyTorch Dataset 클래스

    Args:
        texts (List[Tuple[str, str]]): (문장1, 문장2) 형태의 튜플 리스트
        labels (List[int]): 문장 순서가 맞는지 여부를 나타내는 정수 레이블 (예: 0 또는 1)
        tokenizer (transformers.PreTrainedTokenizer): HuggingFace 토크나이저 객체
        max_length (int): 토큰 최대 길이 (default=128)
    """
    def __init__(self, dataframe, tokenizer, max_length=MAX_TOKEN_LENGTH):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoding = self.tokenizer(
            row['sentence1'],
            row['sentence2'],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(row['label'], dtype=torch.long)
        }

        if 'token_type_ids' in encoding:
            item['token_type_ids'] = encoding['token_type_ids'][0]

        return item

# 3. 모델 아키텍쳐

In [20]:
class SentencePairModel(nn.Module):
    def __init__(self, model_name, num_labels=2, class_weights=None):
        super(SentencePairModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)

        # 1. 더 큰 hidden size 사용
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size * 2, 1024),  # 512 -> 1024
            nn.ReLU(),
            nn.Dropout(0.1),  # dropout 비율 감소
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )

        # 2. 더 많은 attention heads
        self.attention = nn.MultiheadAttention(
            embed_dim=self.bert.config.hidden_size,
            num_heads=12,  # 8 -> 12
            dropout=0.1
        )

        self.layer_norm = nn.LayerNorm(self.bert.config.hidden_size)

        if class_weights is not None:
            self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        else:
            self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids,
                          output_hidden_states=True,
                          return_dict=True)

        # 3. 더 많은 레이어 사용
        hidden_states = outputs.hidden_states
        last_five = torch.stack(hidden_states[-5:])  # 4 -> 5
        avg_hidden = torch.mean(last_five, dim=0)

        avg_hidden = avg_hidden.permute(1, 0, 2)
        attn_output, _ = self.attention(avg_hidden, avg_hidden, avg_hidden)
        attn_output = attn_output.permute(1, 0, 2)

        attn_output = self.layer_norm(attn_output)

        # 4. [CLS]와 마지막 토큰 모두 사용
        cls_output = attn_output[:, 0]
        last_token_output = attn_output[:, -1]
        combined_output = torch.cat([cls_output, last_token_output], dim=-1)

        logits = self.classifier(combined_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {'loss': loss, 'logits': logits}

# 4. 학습 코드

In [21]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [22]:
# 데이터 분할: 학습/검증 (예: 8:2)
train_df, val_df = train_test_split(pairwise_df, test_size=0.2, stratify=pairwise_df['label'], random_state=42)

# dataset 생성
train_dataset = SentencePairDataset(train_df, tokenizer, max_length=MAX_TOKEN_LENGTH)
val_dataset = SentencePairDataset(val_df, tokenizer, max_length=MAX_TOKEN_LENGTH)

In [23]:
train_dataset[0]

{'input_ids': tensor([    2,  7582,  4110,  7696, 10749,  3757,  8466,  4283,  6360,  6485,
          4112,  7202,  6393,  4073,  7899,  4199,  4139,  6542,  4292, 18494,
             3,  6721, 10749, 11090, 14411,  4199,  4139, 16873,  7306,  4112,
          7046,  6745,  4292,  6707,  4279,  4219,  3265,  4050,  4183,  4351,
          4292,  7076,  4034,  2384,  7991,  4519,  2048, 24387,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [24]:
val_dataset[0]

{'input_ids': tensor([    2,  7222,  4234, 14740,  4034,  7222,  4007,  9005,  4283,  6325,
         10728,  6277,  6548,  4073,  7857,  4292,  2064,  4034,  7122,  4292,
          6383,  7796,     3,  6277, 28729,  4034,  6522,  4283,  6246,  4110,
          6686,  4279,  4031,  6311,  6679, 10749,  6417,  4283,  6408,  4047,
          6263,  4073,  6337, 26509, 11041,  4110, 11176,  4114, 15397,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [25]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [26]:
# 모델 인스턴스 생성
model = SentencePairModel("monologg/koelectra-base-v3-discriminator").to(device)

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

In [27]:
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)

    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds)
    }

# 4-1. 하이퍼파라미터 튜닝

In [28]:
# 기본 파라미터 정의
base_params = {
    'logging_dir': './koelectra_logs',
    'logging_steps': 10,
    'save_strategy': 'epoch',
    'eval_strategy': 'epoch',
    'seed': 42,
    'load_best_model_at_end': True,
    'metric_for_best_model': 'eval_loss',  # loss를 기준으로 변경
    'greater_is_better': False,  # loss는 낮을수록 좋으므로 False
    'report_to': 'none',
    'save_safetensors': False,
}

In [29]:
# 팀원 2의 Random Search 설정
param_ranges_2 = {
    'learning_rate': [1e-5, 1e-4],
    'weight_decay': [0.0001, 0.1],
    'warmup_steps': [200, 1000],
    'num_train_epochs': [3, 10],
    'train_batch_size': [8, 16],
    'eval_batch_size': [16, 32],
    'gradient_accumulation_steps': [2, 4]
}

def random_search_team2(n_trials=10):
    results = []

    for trial in range(n_trials):
        # 매 trial마다 다른 시드값 사용
        np.random.seed(trial)
        random.seed(trial)
        # 하이퍼파라미터 샘플링
        lr = loguniform.rvs(param_ranges_2['learning_rate'][0],
                           param_ranges_2['learning_rate'][1])
        wd = loguniform.rvs(param_ranges_2['weight_decay'][0],
                           param_ranges_2['weight_decay'][1])
        warmup = int(np.random.uniform(param_ranges_2['warmup_steps'][0],
                                     param_ranges_2['warmup_steps'][1]))
        epochs = int(np.random.uniform(param_ranges_2['num_train_epochs'][0],
                                     param_ranges_2['num_train_epochs'][1]))
        train_batch = int(np.random.uniform(param_ranges_2['train_batch_size'][0],
                                          param_ranges_2['train_batch_size'][1]))
        eval_batch = int(np.random.uniform(param_ranges_2['eval_batch_size'][0],
                                         param_ranges_2['eval_batch_size'][1]))
        grad_acc = int(np.random.uniform(param_ranges_2['gradient_accumulation_steps'][0],
                                       param_ranges_2['gradient_accumulation_steps'][1]))

        # GPU 최적화 설정
        gpu_optimizations = {
            'fp16': True,
            #'gradient_checkpointing': True
        }

        # TrainingArguments 설정
        current_args = TrainingArguments(
            output_dir=f'./koelectra_results/team2_trial{trial+1}',
            learning_rate=lr,
            weight_decay=wd,
            warmup_steps=warmup,
            per_device_train_batch_size=train_batch,
            per_device_eval_batch_size=eval_batch,
            num_train_epochs=epochs,
            gradient_accumulation_steps=grad_acc,
            lr_scheduler_type=np.random.choice(['linear', 'cosine']),
            **base_params,
            **gpu_optimizations
        )

        # 모델 학습 및 평가
        trainer = Trainer(
            model=model,
            args=current_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        print(f"Trial {trial+1} - Learning Rate: {lr}, Weight Decay: {wd}, Warmup Steps: {warmup}, Train Batch Size: {train_batch}, Eval Batch Size: {eval_batch}, Epochs: {epochs}, Gradient Accumulation Steps: {grad_acc}")

        trainer.train()
        eval_results = trainer.evaluate()

        # 결과 저장
        results.append({
            'trial': trial + 1,
            'learning_rate': lr,
            'weight_decay': wd,
            'warmup_steps': warmup,
            'train_batch_size': train_batch,
            'eval_batch_size': eval_batch,
            'epochs': epochs,
            'gradient_accumulation_steps': grad_acc,
            'eval_accuracy': eval_results['eval_accuracy'],
            'eval_loss': eval_results['eval_loss']
        })

        print(f"Trial {trial+1} 정확도: {eval_results['eval_accuracy']:.4f} & 손실: {eval_results['eval_loss']:.4f}")

    return pd.DataFrame(results)

In [30]:
'''실제 하이퍼파라미터 튜닝 실행 코드'''

print("Random Search 시작")
results = random_search_team2() # 팀원에 따라 함수 변경
results.to_csv('random_search_results2.csv', index=False)   # 팀원에 따라 파일 이름 변경

Random Search 시작
Trial 1 - Learning Rate: 3.5384535909065495e-05, Weight Decay: 0.013981961408994055, Warmup Steps: 682, Train Batch Size: 11, Eval Batch Size: 26, Epochs: 6, Gradient Accumulation Steps: 2


model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.492200,0.492515,0.750000,0.000000,0.000000,0.000000
2,0.367100,0.446447,0.806667,0.553846,0.654545,0.480000
3,0.253800,0.361483,0.852917,0.683408,0.739806,0.635000
4,0.155200,0.346852,0.885417,0.755120,0.810707,0.706667
5,0.023800,0.568504,0.893750,0.778068,0.814208,0.745000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Trial 1 정확도: 0.8854 & 손실: 0.3469
Trial 2 - Learning Rate: 2.6122937099604327e-05, Weight Decay: 0.01448683388723935, Warmup Steps: 200, Train Batch Size: 9, Eval Batch Size: 17, Epochs: 5, Gradient Accumulation Steps: 2


...

Trial 8 정확도: 0.8838 & 손실: 0.4475
Trial 9 - Learning Rate: 7.471871645028575e-05, Weight Decay: 0.08046777417892739, Warmup Steps: 895, Train Batch Size: 9, Eval Batch Size: 16, Epochs: 6, Gradient Accumulation Steps: 2


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.073300,0.930094,0.880833,0.758037,0.769759,0.746667
2,0.092100,0.850707,0.843333,0.639155,0.753394,0.555000
3,0.144100,0.472696,0.881667,0.752182,0.789377,0.718333
4,0.054400,0.514674,0.895417,0.771194,0.851107,0.705000
5,0.000500,0.693165,0.901667,0.794425,0.832117,0.760000


Trial 9 정확도: 0.8817 & 손실: 0.4727
Trial 10 - Learning Rate: 1.0241749607088557e-05, Weight Decay: 0.0032034929786131467, Warmup Steps: 596, Train Batch Size: 9, Eval Batch Size: 19, Epochs: 3, Gradient Accumulation Steps: 2


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.076300,0.631435,0.895000,0.786802,0.798969,0.775000
2,0.000200,0.707656,0.898750,0.795970,0.802030,0.790000


Trial 10 정확도: 0.8950 & 손실: 0.6314


In [ ]:
# 모든 결과를 하나의 DataFrame으로 합치기
result1 = pd.read_csv('random_search_results1.csv')
result2 = pd.read_csv('random_search_results2.csv')
result3 = pd.read_csv('random_search_results3.csv')
result4 = pd.read_csv('random_search_results4.csv')
result5 = pd.read_csv('random_search_results5.csv')
all_results_df = pd.concat([result1, result2, result3, result4, result5], ignore_index=True)

# 결과를 정확도 기준으로 정렬
all_results_df = all_results_df.sort_values('eval_accuracy', ascending=False)

# 최적의 하이퍼파라미터 조합 출력
print("\n최적의 하이퍼파라미터 조합:")
print(all_results_df.iloc[0])

# 각 팀원별 최고 성능 조합 출력
print("\n팀원별 최고 성능 조합:")
for team_id in range(1, 6):
    team_best = all_results_df[all_results_df['team_member'] == team_id].iloc[0]
    print(f"\n팀원 {team_id}의 최고 성능 조합:")
    print(team_best)

# 결과를 CSV 파일로 저장
all_results_df.to_csv('random_search_results.csv', index=False)

# 5. 평가 및 추론

In [ ]:
def predict_order(sent1, sent2, model, tokenizer, device='cpu'):
    model.eval()
    inputs = tokenizer(
        sent1,
        sent2,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=MAX_TOKEN_LENGTH
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs['logits'], dim=1)

    return predictions, predictions.argmax().item()

# 6. 페어와이즈 예측 함수

In [ ]:
def pair_score(s1, s2):
    probs, _ = predict_order(s1, s2, model, tokenizer, device)
    return probs[0][1].item()  # label=1 확률

# 7. 순열별 점수 계산 후 최적 순서 선택

In [ ]:
pred_orders = []
pred_scores = []

# 테스트 샘플 수 만큼 반복
for idx, (_, row) in enumerate(tqdm(test.iterrows(),
                                     total=len(test),
                                     desc="문장 순서 예측")):
    sents = [row[f'sentence_{i}'] for i in range(4)]

    best_perm, best_score = None, -1.0
    # 24가지 순열 전부 탐색
    for perm in permutations(range(4), 4):
        # 인접 페어 3개의 점수 합산
        score = sum(pair_score(sents[perm[i]], sents[perm[i+1]])
                    for i in range(3))
        if score > best_score:
            best_score, best_perm = score, perm

    pred_orders.append(best_perm)
    pred_scores.append(best_score)

    # # 매 50개마다 로그 출력
    # if (idx + 1) % 50 == 0:
    #     print(f"[{idx+1}/{len(test)}] 현재까지 최고 평균 스코어: {np.mean(pred_scores):.4f}")

# 8. submission 파일 작성

In [ ]:
for idx, perm in enumerate(pred_orders):
    for i, p in enumerate(perm):
        submission.loc[idx, f'answer_{i}'] = p

submission.to_csv('koelectra_results/submission.csv', index=False)
print("✅ submission.csv 파일이 생성되었습니다.")